In [14]:
from torchvision import transforms, datasets
import numpy as np
import time

In [15]:
train = datasets.MNIST("", train = True, download = True,
                       transform = transforms.Compose([transforms.ToTensor()]))
test = datasets.MNIST("", train = False, download = True,
                       transform = transforms.Compose([transforms.ToTensor()]))

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [16]:
X_train = train.data
X_test = test.data
y_train = train.targets
y_test = test.targets

X_train = X_train.reshape(60000,784).numpy()
X_test = X_test.reshape(10000, 784).numpy()
y_train = y_train.numpy()
y_test = y_test.numpy()

X_train = X_train / 255
X_test = X_test / 255 

NameError: name 'train' is not defined

In [ ]:
zero = [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
one = [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]
two = [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]
three = [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]
four = [0., 0., 0., 0., 1.,0., 0., 0., 0., 0.]
five = [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]
six = [0., 0., 0., 0., 0., 0., 1.,0., 0., 0.]
seven = [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]
eight = [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]
nine = [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]

In [ ]:
y_test_new = []

for i in range(len(y_test)):
    if y_test[i] == 0:
        y_test_new.append(zero)
    elif y_test[i] == 1:
        y_test_new.append(one)
    elif y_test[i] == 2:
        y_test_new.append(two)
    elif y_test[i] == 3:
        y_test_new.append(three)
    elif y_test[i] == 4:
        y_test_new.append(four)
    elif y_test[i] == 5:
        y_test_new.append(five)
    elif y_test[i] == 6:
        y_test_new.append(six)
    elif y_test[i] == 7:
        y_test_new.append(seven)
    elif y_test[i] == 8:
        y_test_new.append(eight)
    elif y_test[i] == 9:
        y_test_new.append(nine)
    
y_train_new = []

for i in range(len(y_train)):
    if y_train[i] == 0:
        y_train_new.append(zero)
    elif y_train[i] == 1:
        y_train_new.append(one)
    elif y_train[i] == 2:
        y_train_new.append(two)
    elif y_train[i] == 3:
        y_train_new.append(three)
    elif y_train[i] == 4:
        y_train_new.append(four)
    elif y_train[i] == 5:
        y_train_new.append(five)
    elif y_train[i] == 6:
        y_train_new.append(six)
    elif y_train[i] == 7:
        y_train_new.append(seven)
    elif y_train[i] == 8:
        y_train_new.append(eight)
    elif y_train[i] == 9:
        y_train_new.append(nine)

In [ ]:
Y_train = np.array(y_train_new)
Y_test = np.array(y_test_new)

In [11]:
class NeuralNetwork():
    
    def __init__(self, sizes=[784, 64, 64, 10], activation_functions=['relu', 'sigmoid', 'softmax'], epochs=50, l_rate=0.01):
        self.sizes = sizes
        self.epochs = epochs
        self.l_rate = l_rate
        self.activation_functions = activation_functions

        # we save all parameters in the neural network in this dictionary
        self.params = self.initialization()

    def sigmoid(self, x, derivative=False):
        if derivative:
            return (np.exp(-x))/((np.exp(-x)+1)**2)
        return 1/(1 + np.exp(-x))

    def softmax(self, x, derivative=False):
        # Numerically stable with large exponentials
        exps = np.exp(x - x.max())
        if derivative:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
        return exps / np.sum(exps, axis=0)
    
    def relu(self, x, derivative=False):
            if derivative:
                copy = np.array(x, copy = True)
                copy[x <= 0] = 0
                copy[x > 0] = 1
                return copy
            return np.maximum(0,x)

    def initialization(self):
        
        layers = []
        params = {}
        for index in range(len(self.sizes)):
            if index != (len(self.sizes)-1):
            
                params['W' + str(index + 1)] = np.random.randn(self.sizes[index+1], self.sizes[index])* np.sqrt(1. / self.sizes[index+1])
        
        return params

    def forward_pass(self, x_train):
        params = self.params
        activation_function = self.activation_functions
        params['A0'] = x_train
        # input layer activations becomes sample
        for index in range(len(self.sizes)):
            if(index != (len(self.sizes)-1)):
            
                if activation_function[index] == 'sigmoid':
                    activation = self.sigmoid
                elif activation_function[index] == 'relu':
                    activation = self.relu
                elif activation_function[index] == 'softmax':
                    activation = self.softmax
                else:
                    print('Activation function not supported')
                
                params['Z' + str(index+1)] = np.dot(params['W' + str(index+1)], params['A' + str(index)])
                params['A' + str(index+1)] = activation(params['Z' + str(index+1)])
        
        # Returns last value -> Last A-value
        last_value = str(list(params)[-1])
        return params[last_value]

    def backward_pass(self, y_train, output):
        
        params = self.params
        activation_function = self.activation_functions
        change_w = {}
        
        for index in reversed(range(len(self.sizes))):
            if(index != 0):
                if activation_function[index-1] == 'sigmoid':
                    activation = self.sigmoid
                elif activation_function[index-1] == 'relu':
                    activation = self.relu
                elif activation_function[index-1] == 'softmax':
                    activation = self.softmax
                else:
                    print('Activation function not supported')
                
                if index == (len(self.sizes) - 1):    
                    error = 2 * (output - y_train) / output.shape[0] * activation(params['Z' + str(index)], derivative=True)
                    change_w['W' + str(index)] = np.outer(error, params['A' + str(index-1)])
                else:
                    error = np.dot(params['W' + str(index+1)].T, error) * activation(params['Z' + str(index)], derivative=True)
                    change_w['W' + str(index)] = np.outer(error, params['A' + str(index-1)])

        return change_w

    def update_network_parameters(self, changes_to_w):
        
        for key, value in changes_to_w.items():
            self.params[key] -= self.l_rate * value

    def compute_accuracy(self, x_val, y_val):

        predictions = []

        for x, y in zip(x_val, y_val):
            output = self.forward_pass(x)
            pred = np.argmax(output)
            predictions.append(pred == np.argmax(y))
        
        return np.mean(predictions)

    def train(self, x_train, y_train, x_val, y_val):
        start_time = time.time()
        for iteration in range(self.epochs):
            for x,y in zip(x_train, y_train):
                output = self.forward_pass(x)
                changes_to_w = self.backward_pass(y, output)
                self.update_network_parameters(changes_to_w)
            
            accuracy = self.compute_accuracy(x_val, y_val)
            print('Epoch: {0}, Time Spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
                iteration+1, time.time() - start_time, accuracy * 100
            ))

In [12]:
dnn = NeuralNetwork()
dnn.train(X_train, Y_train, X_test, Y_test)

Epoch: 1, Time Spent: 47.13s, Accuracy: 66.90%
Epoch: 2, Time Spent: 89.79s, Accuracy: 79.50%
Epoch: 3, Time Spent: 132.26s, Accuracy: 88.05%
Epoch: 4, Time Spent: 178.97s, Accuracy: 89.63%
Epoch: 5, Time Spent: 222.51s, Accuracy: 90.76%
Epoch: 6, Time Spent: 270.19s, Accuracy: 91.59%
Epoch: 7, Time Spent: 329.54s, Accuracy: 92.27%
Epoch: 8, Time Spent: 383.52s, Accuracy: 92.75%
Epoch: 9, Time Spent: 445.19s, Accuracy: 93.20%
Epoch: 10, Time Spent: 503.50s, Accuracy: 93.51%
Epoch: 11, Time Spent: 555.19s, Accuracy: 93.84%
Epoch: 12, Time Spent: 608.43s, Accuracy: 94.14%
Epoch: 13, Time Spent: 658.52s, Accuracy: 94.42%
Epoch: 14, Time Spent: 710.75s, Accuracy: 94.67%
Epoch: 15, Time Spent: 766.83s, Accuracy: 94.84%
Epoch: 16, Time Spent: 824.19s, Accuracy: 95.02%
Epoch: 17, Time Spent: 881.01s, Accuracy: 95.13%
Epoch: 18, Time Spent: 939.62s, Accuracy: 95.29%
Epoch: 19, Time Spent: 995.71s, Accuracy: 95.39%
Epoch: 20, Time Spent: 1047.04s, Accuracy: 95.54%
Epoch: 21, Time Spent: 1093.53